In [1]:
import os
import pandas as pd

In [5]:
# Путь к папке, где лежат dish_00000, dish_00001 и т.д.
base_dir = "C:/Users/Leopard/Desktop/Jupyter_Python_ML/Univer/Final_AI_model/Dataset_dish"

In [7]:
# === Шаг 1. Загрузка CSV без заголовков ===
meta1 = pd.read_csv("C:/Users/Leopard/Desktop/Jupyter_Python_ML/Univer/Final_AI_model/dataset_fianl/dish_metadata_cafe1.csv",
                    header=None,
                    engine='python',
                    on_bad_lines='skip')

meta2 = pd.read_csv("C:/Users/Leopard/Desktop/Jupyter_Python_ML/Univer/Final_AI_model/dataset_fianl/dish_metadata_cafe2.csv",
                    header=None,
                    engine='python',
                    on_bad_lines='skip')

In [9]:
# === Шаг 2. Объединение ===
meta = pd.concat([meta1, meta2], ignore_index=True)

In [11]:
# === Шаг 3. Назначаем названия колонок ===
# Первая колонка — dish_id, вторая — total_calories
# Остальные пока не нужны, но дадим им временные имена
meta.columns = ['dish_id', 'total_calories'] + [f'col_{i}' for i in range(meta.shape[1] - 2)]

In [13]:
# Категоризация по калориям
bins = [0, 300, 600, meta['total_calories'].max() + 1]
labels = ['low', 'medium', 'high']
meta['cal_label'] = pd.cut(meta['total_calories'], bins=bins, labels=labels)

In [15]:
print(meta[['dish_id', 'total_calories', 'cal_label']].head())

           dish_id  total_calories cal_label
0  dish_1561662216      300.794281    medium
1  dish_1562688426      137.569992       low
2  dish_1561662054      419.438782    medium
3  dish_1562008979      382.936646    medium
4  dish_1560455030       20.590000       low


In [17]:
meta['dish_id'] = meta['dish_id'].str.replace('dish_', '').astype('int64')

In [52]:
# Создаём список фото (берём по 1 фото из каждой папки)
records = []

for folder in os.listdir(base_dir):
    folder_path = os.path.join(base_dir, folder)
    if os.path.isdir(folder_path) and folder.startswith("dish_"):
        dish_id = folder.split("_")[1]
        frames_folder = os.path.join(folder_path, "frames_sampled30")
        if os.path.isdir(frames_folder):
            images = [f for f in sorted(os.listdir(frames_folder))
                      if os.path.isfile(os.path.join(frames_folder, f))]
            if images:
                first_image_path = os.path.join(frames_folder, images[0])
                records.append({'dish_id': int(dish_id), 'image_path': first_image_path})

# В DataFrame
df_images = pd.DataFrame(records)

# Объединяем с метками калорий
df_full = df_images.merge(meta[['dish_id', 'cal_label']], on='dish_id')

# Сохраняем
df_full.to_csv('image_labels.csv', index=False)
print("Готово! Сохранено в image_labels.csv")

Готово! Сохранено в image_labels.csv
